In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '.notebook-container { width:100% !important;} '
        '.container { width:100% !important;} '
    '</style>'
))


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
from wurlitzer import sys_pipes
from plotnine import *
from mizani.formatters import percent_format
import itertools
import pandas as pd
import numpy as np

import UCP.input.parser as ucp_parser
import generic.optimization.solution_extraction as ucp_out
from UCP.model.ucp import create_model, extract_solution_kpi
from UCP.relaxations.lagrangian.production_state.relaxation import (
    ProductionStateRelaxation,
)
from UCP.relaxations.lagrangian.demand_satisfaction import DemandSatRelaxation
from UCP.relaxations.lagrangian.heuristic import combinatorial_heuristic

from generic.series_dict import (
    series_dict_into_array,
    series_dict_to_array,
    array_into_series_dict,
    allocate_array_for,
    full_series_dict
)
from generic.optimization.model import OptimizationSense
from generic.optimization.dual_optimization.lagrangian_decomposition import (
    multipliers_range,
)

from generic.optimization.dual_optimization.algorithms.cutting_plane import (
    CuttingPlane, ColumnGeneration
)

from generic.optimization.dual_optimization.algorithms.subgradient import (
    ComposableSubgradientMethod,
    PolyakStepSizeRule,
    CFMDeflection,
    FixedDeflection,
    CFMDeflection,
    FixedTargetTracker,
)

from generic.optimization.dual_optimization.algorithms.subgradient.volume import (
    VolumeAlgorithm,
    BundleVolumeAlgorithm,
)

import UCP.output.check_solution as ck
from generic.optimization.solution_extraction import extract_solution
from UCP.heuristics.relax_and_opt import relax_and_opt
from UCP.output.charts import total_production, enp_vs_eie

In [4]:
theme_set(theme_bw() + theme(figure_size=(10, 10 / 1.61)))

In [5]:
data = ucp_parser.read_instance("./UCP/data/instance_5g.ucp")
model = create_model(data)

In [6]:
relaxation = DemandSatRelaxation(data)

var_lb, var_ub = tuple(
    series_dict_to_array(**m)
    for m in multipliers_range(relaxation.dualized_constraints)
)

lower_cost_estimate = data.loads["value"].sum() * data.thermal_plants["l_cost"].min()
upper_cost_estimate = data.loads["value"].sum() * data.c_ENP


In [8]:
upper_bound = np.nan
current_target = np.nan

primal_bound = upper_cost_estimate
primal_solution = None
best_dual_bound = relaxation.sense * np.infty
cp_primal_bound = relaxation.sense * -np.infty


def run_heuristic(
    primal_solutions,
    data,
):
    commitments = [ps["s"] for ps in primal_solutions]
    fixed_model, _ = combinatorial_heuristic(
        data, commitments, combination_options={"mip": 0}
    )
    solution = extract_solution(fixed_model)
    solution_kpis = extract_solution_kpi(fixed_model, data, solution)
    print(
        f" => Heuristic:\tTotal cost: {solution_kpis['objective']:6.3g}\t"
        + f"Demand mismatch cost:{solution_kpis['demand_mismatch_costs'].sum():6.3g}"
    )

    intercept = fixed_model.objs["total_production_cost"].value()
    subgradient = relaxation.subgradient(solution)

    return intercept, subgradient, solution, solution_kpis["objective"]


fixed_tracker = FixedTargetTracker(sense=-relaxation.sense, overestimation_factor=0.2)


def target_tracker(current_value: float) -> float:
    if (
        np.isnan(upper_bound)
        or abs(upper_bound - current_value) / abs(current_value + 0.1) > 0.2
    ):
        fixed_target = fixed_tracker(current_value)
        result = max(fixed_target, lower_cost_estimate)
    else:
        result = upper_bound
    current_target = result
    return result


sgd_method = ComposableSubgradientMethod(
    sense=-relaxation.sense,
    var_lb=var_lb,
    var_ub=var_ub,
    step_size_fun=PolyakStepSizeRule(-relaxation.sense, target_tracker),
    deflection_fun=CFMDeflection(),
)

vol_method = VolumeAlgorithm(
    sense=-relaxation.sense,
    var_lb=var_lb,
    var_ub=var_ub,
    step_size_fun=PolyakStepSizeRule(-relaxation.sense, target_tracker),
)

bvol_method = BundleVolumeAlgorithm(
    sense=-relaxation.sense,
    var_lb=var_lb,
    var_ub=var_ub,
    step_size_fun=PolyakStepSizeRule(-relaxation.sense, target_tracker),
)

cp = CuttingPlane(sense=-relaxation.sense, var_lb=var_lb, var_ub=var_ub)

it = 0
max_iter = 60
heur_freq = 5

multipliers = full_series_dict(
    0.0, **{k: v.index for k, v in relaxation.dualized_constraints.items()}
)
new_multipliers = full_series_dict(
    0.0, **{k: v.index for k, v in relaxation.dualized_constraints.items()}
)

vect_subgradient = allocate_array_for(**multipliers)
vect_multipliers = allocate_array_for(**multipliers)
new_vect_multipliers = allocate_array_for(**multipliers)

ph_vect_subgradient = allocate_array_for(**multipliers)

primal_solutions = []
algo_kpis = []

sgd_algorithms = {
    "standard": sgd_method,
    "volume": vol_method,
    "bundle_volume": bvol_method,
}

sgd_algorithm = "bundle_volume"
sgd_alg = sgd_algorithms[sgd_algorithm]

sgd_counter = 0
cp_counter = 0
sgd_max_it = 10
cp_max_it = 10
sgd_flag = True

algorithm = ""


while True:
    kpi = {}

    evaluation = relaxation.evaluate(multipliers, mip=1)
    primal_solutions.append(evaluation.primal_solution)

    series_dict_into_array(vect_subgradient, **evaluation.subgradient)
    series_dict_into_array(vect_multipliers, **multipliers)

    cp.add_cut(vect_subgradient, evaluation.original_objective)
    
    if sgd_counter >= sgd_max_it:
        sgd_flag = False
        sgd_counter = 0
    if cp_counter >= cp_max_it:
        sgd_flag = True
        cp_counter = 0
        sgd_alg.restart()

    if sgd_flag:
        new_vect_multipliers = sgd_alg(evaluation.value, vect_subgradient, vect_multipliers)
        sgd_counter += 1
        algorithm = sgd_algorithm
    else:
        cp_counter += 1
        cp_primal_bound, new_vect_multipliers = cp.solve(gapRel=0.05, mip=0, options=["barrier", "crossover", "0"])
        kpi["cp_primal_bound"] = cp_primal_bound
        algorithm = "CP"

    
    array_into_series_dict(new_vect_multipliers, **new_multipliers)

    kpi["multipliers"] = vect_multipliers
    kpi["subgradient"] = vect_subgradient
    kpi["dual_bound"] = evaluation.value

    if relaxation.sense * (best_dual_bound - evaluation.value) > 0:
        best_dual_bound = evaluation.value

    if it > 0 and it % heur_freq == 0:
        intercept, subgradient, solution, ph_value = run_heuristic(
            primal_solutions, data
        )
        if primal_bound > ph_value:
            primal_bound = ph_value
            primal_solutions.append(solution)
            upper_bound = primal_bound
            kpi["primal_bound"] = primal_bound

        series_dict_into_array(ph_vect_subgradient, **subgradient)
        cp.add_cut(vect_subgradient, intercept)
    
    algo_kpis.append(kpi)

    gap = abs(upper_bound - best_dual_bound) / (abs(upper_bound)+1e-4)
    cp_gap = abs(cp_primal_bound - best_dual_bound)/(abs(cp_primal_bound)+1e-4)
    mult_change = np.linalg.norm(vect_multipliers - new_vect_multipliers)
    mult_norm = np.linalg.norm(vect_multipliers)
    subgrd_norm = np.linalg.norm(vect_subgradient)

    print(
        f"It:{it}\tAlgorithm:{algorithm}\tbest dual bound: {best_dual_bound:6.4g},\t"
        + f"best primal: {upper_bound:6.4g}\tgap: {gap*100:5.2f}%\tCP gap:{cp_gap*100:5.2f}%\n"
        + f"  => mult. norm: {mult_norm:6.4g},\t subgrd. norm: {subgrd_norm:6.4g},\tmult. change:{mult_change:5.2g}"
    )

    it += 1
    multipliers = new_multipliers

    if gap < 0.01 or it > max_iter:
        break

<ipython-input-8-aa8fabfb7729>:164: RuntimeWarning: invalid value encountered in double_scalars


It:0	Algorithm:bundle_volume	best dual bound:      0,	best primal:    nan	gap:   nan%	CP gap:  nan%
  => mult. norm:      0,	 subgrd. norm:   4210,	mult. change:   20
It:1	Algorithm:bundle_volume	best dual bound: 8.417e+04,	best primal:    nan	gap:   nan%	CP gap:  nan%
  => mult. norm:  19.99,	 subgrd. norm:   4210,	mult. change:    2
It:2	Algorithm:bundle_volume	best dual bound: 9.258e+04,	best primal:    nan	gap:   nan%	CP gap:  nan%
  => mult. norm:  21.99,	 subgrd. norm:   4210,	mult. change:  2.2
It:3	Algorithm:bundle_volume	best dual bound: 1.011e+05,	best primal:    nan	gap:   nan%	CP gap:  nan%
  => mult. norm:  24.19,	 subgrd. norm:   3836,	mult. change:  2.4
It:4	Algorithm:bundle_volume	best dual bound: 1.089e+05,	best primal:    nan	gap:   nan%	CP gap:  nan%
  => mult. norm:  26.59,	 subgrd. norm:   2943,	mult. change:  2.8
[]
 => Heuristic:	Total cost: 3.26e+06	Demand mismatch cost:3.21e+06
It:5	Algorithm:bundle_volume	best dual bound: 1.169e+05,	best primal: 3.256e+06	gap:

It:46	Algorithm:bundle_volume	best dual bound: 1.356e+05,	best primal: 1.455e+05	gap:  6.82%	CP gap: 4.51%
  => mult. norm:  59.41,	 subgrd. norm:   2789,	mult. change:    0
It:47	Algorithm:bundle_volume	best dual bound: 1.356e+05,	best primal: 1.455e+05	gap:  6.82%	CP gap: 4.51%
  => mult. norm:  59.41,	 subgrd. norm:   2789,	mult. change:   24
It:48	Algorithm:bundle_volume	best dual bound: 1.356e+05,	best primal: 1.455e+05	gap:  6.82%	CP gap: 4.51%
  => mult. norm:  46.48,	 subgrd. norm:   2302,	mult. change:  9.4
It:49	Algorithm:bundle_volume	best dual bound: 1.356e+05,	best primal: 1.455e+05	gap:  6.82%	CP gap: 4.51%
  => mult. norm:  43.97,	 subgrd. norm:   2597,	mult. change:   28
[]
 => Heuristic:	Total cost: 2.82e+05	Demand mismatch cost:1.43e+05
It:50	Algorithm:CP	best dual bound: 1.356e+05,	best primal: 1.455e+05	gap:  6.82%	CP gap: 3.55%
  => mult. norm:  37.34,	 subgrd. norm:   2631,	mult. change:   89
It:51	Algorithm:CP	best dual bound: 1.356e+05,	best primal: 1.455e+05	ga

In [ ]:
iteration_info = pd.DataFrame.from_records(algo_kpis)
iteration_info

In [ ]:
temp = pd.melt(
    iteration_info[
        ["iteration", "dual_bound", "best_dual_bound", "cp_best_primal_bound"]
    ],
    id_vars=["iteration"],
)

best_dual = iteration_info["best_dual_bound"].max()
best_primal = iteration_info["cp_best_primal_bound"].min()

delta = max(abs(best_dual), abs(best_primal)) * 0.25
ggplot(
    temp, aes(x="iteration", y="value", color="variable", shape="variable")
) + geom_line() + geom_point() + coord_cartesian(ylim=(best_dual - delta, best_primal + delta))

In [ ]:
from sklearn.manifold import TSNE, MDS 
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA, SparsePCA, FastICA, NMF

In [ ]:
vect_matrix = np.vstack(iteration_info["multipliers_vector"][4:].to_list())
vect_matrix.shape

X=PCA(n_components=3).fit_transform(StandardScaler().fit_transform(vect_matrix))
df = pd.DataFrame(X, columns=[f"x{i}" for i in range(X.shape[1])]).reset_index().rename(columns={"index":"iteration"})
df["iteration"]+=4
df = pd.merge(df, iteration_info[["iteration", "dual_bound"]])

temp = pd.melt(df, id_vars=["iteration", "dual_bound", "x0"])

delta_y = (temp["value"].max()- temp["value"].min())/40
delta_x = (temp["x0"].max()- temp["x0"].min())/40
temp["label_pos_y"]=temp["value"]+delta_y
temp["label_pos_x"]=temp["x0"]+delta_x
ggplot(temp, aes("x0", "value", color="dual_bound")) \
+ geom_point(size=2) \
+ geom_text(aes(label="iteration",x="label_pos_x", y="label_pos_y"), position=position_jitter(width=0.4), color="black") + facet_wrap("~variable")

In [ ]:
solution = dual_bounds_list[-1].primal_solution

In [ ]:
solution["s"].groupby(level="plant").sum()

In [ ]:
solution["p"].groupby(level="period").sum()

In [ ]:
tot_production = solution["p"].groupby(level=["period"]).sum().reset_index()

tot_production = pd.merge(tot_production, data.loads[["period", "value"]], on="period")

tot_production = tot_production.rename(columns={"value": "load"})
tot_production = pd.melt(tot_production, id_vars=["period"])
ggplot(tot_production, aes("period", "value", color="variable", linetype="variable")) \
    + geom_step(size=2) \
    + scale_color_discrete(name="Series", breaks=["load", "p"], labels=["Load", "Production"]) \
    + scale_linetype_discrete(name="Series", breaks=["load", "p"], labels=["Load", "Production"]) \
    + labs(y="Value [MW]") \
    + ggtitle("Production vs Load")

In [ ]:
demand_gap = pd.DataFrame(EIE=solution["EIE"], ENP=solution["ENP"])
demand_gap = pd.melt(demand_gap, id_vars="period", var_name="Series")

ggplot(demand_gap, aes(x="period", y="value", color="Series", linetype="Series")) \
    + geom_step(size=2) \
    + labs(x="period", y="Value [MW]") \
    + ggtitle("Demand mismatch")

In [ ]:
ggplot(solution["p"], aes("period", "production")) \
    + geom_step() \
    + facet_wrap("~plant", labeller=lambda s: f"plant #{s}") \
    + labs(x="hour", y="production [MW]") \
    + ggtitle("Hourly production per plant")

In [ ]:
avg_coef = data.thermal_plants.copy()
avg_coef["avg_coef"] = avg_coef.l_cost + (avg_coef.c_cost / avg_coef.max_power)

utilization = solution["p"].groupby("plant").agg({"production": "sum"}).reset_index(0)

utilization = utilization.merge(data.thermal_plants[["plant", "max_power"]])

utilization["utilization"] = utilization.production / (utilization.max_power * len(data.time))

temp = avg_coef[["plant", "avg_coef"]]\
    .merge(utilization, sort=True)\
    [["plant", "utilization", "avg_coef"]]

ggplot(temp, aes("avg_coef", "utilization")) \
    + geom_point(size=2) \
    + geom_text(aes(y="utilization+0.04", label="plant")) \
    + scale_y_continuous(labels=percent_format()) \
    + labs(x="Avg. Hourly cost [€/MW]", y="Utilization %", label="Plant id") \
    + ggtitle("Utilization vs Hourly cost")

In [ ]:
from pulp import *
import sys
print(f"==> PuLP version:")
!{sys.executable} -m pip show pulp

problem = LpProblem(sense=LpMaximize)

z = LpVariable("z")
x = LpVariable("x",2,5)
y = LpVariable("y", 1,3)

problem.addConstraint(z+x+0.0*y <=1)

print("\n==>Solving problem as created in PuLP")
problem.writeLP("problem.lp")
problem.solve()

print("=>PuLP solution")
print(f"z={z.value()}")
print(f"x={x.value()}")
print(f"y={y.value()} <-- PROBLEM: this one is going to be None")

print("\n")

print("\n==> Solving problem manually written in LP format")
!cat problem2.lp
print("Calling CBC")
!cbc problem2.lp solve solu solution.txt
print("Solution computed by CBC")
!cat solution.txt